In [1]:
# This notebook checks MNE functions for ECG and EOG detection
# Cropped data is used here (5 minutes only), tried on whole data - takes forever.


#Load data, make folders
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
import configparser
import ancpbids

from main_meg_qc import make_derivative_html, sanity_check, initial_stuff
from RMSE_meq_qc import RMSE_meg_qc
from PSD_meg_qc import PSD_meg_qc
from Peaks_auto_meg_qc import PP_auto_meg_qc


In [ ]:
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'

raw = mne.io.read_raw_fif(data_file)

raw

In [2]:
#%% TRY TO SAVE DERIVATIVE FROM BIDS DATASET HERE:
config_file_name = 'settings.ini'
make_derivative_html(config_file_name)

Opening raw data file /Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif...
    Range : 60000 ... 1255999 =     60.000 ...  1255.999 secs
Ready.
stimulus ['STI101', 'STI201', 'STI301']
182 events found
Event IDs: [2 4 8]
Trigger channel has a non-zero initial value of 3840 (consider using initial_event=True to detect this event)
Not setting metadata
182 matching events found
No baseline correction applied
0 projection items activated
Loading data for 182 events and 1201 original time points ...
0 bad epochs dropped
Not setting metadata
182 matching events found
No baseline correction applied
0 projection items activated
Loading data for 182 events and 1201 original time points ...
0 bad epochs dropped


OSError: Cannot save file into a non-existent directory: '../derivatives/sub-009/megqc/csv files'

In [ ]:
# Other useful ancp stuff:

config = configparser.ConfigParser()
config.read('settings.ini')

default_direct = config['DEFAULT']['data_directory']
dataset_path = ancpbids.utils.fetch_dataset(default_direct)

from ancpbids import BIDSLayout
layout = BIDSLayout(dataset_path)

list_of_fifs = layout.get(suffix='meg', extension='.fif', return_type='filename')

list_of_subs = layout.get_subjects()

list_of_entities = layout.get_entities()
print(list_of_entities)

In [ ]:
#%% TRY SEPARATE FUNCTIONS HERE

config = configparser.ConfigParser()
config.read('settings.ini')
data_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-1.fif'
#data_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/ds004276/sub-001/meg/sub-001_task-words_meg.fif'
sid='001'

df_epochs, epochs_mg, channels, raw_bandpass, raw_bandpass_resamp, raw_cropped, raw = initial_stuff(config, data_file)
m_or_g_chosen = ['mags', 'grads']

m_or_g_chosen = sanity_check(m_or_g_chosen, channels)

if len(m_or_g_chosen) == 0: 
    raise ValueError('No channels to analyze. Check presence of mags and grads in your data set and parameter do_for in settings.')




In [ ]:
dfs_ptp_amlitude_annot, bad_channels, amplit_annot_with_ch_names = PP_auto_meg_qc(sid, config, channels, raw_bandpass_resamp, m_or_g_chosen)

#list_of_figures, _, list_of_fig_desc = RMSE_meg_qc(sid, config, channels, df_epochs, raw_bandpass_resamp, m_or_g_chosen)

#from PSD_meg_qc import Freq_Spectrum_meg

#freqs, psds, fig_path_psd, fig_psd, fig_desc = Freq_Spectrum_meg(data=raw_bandpass_resamp, m_or_g = 'mags', sid=sid, freq_min=0.5, freq_max=100, n_fft=1000, n_per_seg=1000, freq_tmin=None, freq_tmax=None, ch_names=channels['mags'])



In [ ]:
dfs_ptp_amlitude_annot

bad_channels

amplit_annot_with_ch_names